In [ ]:
weather_filepath = 'https://raw.githubusercontent.com/leekyuyoung20230313/pythone/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/data/weather_33.csv'

In [ ]:
rides_path = '/content/drive/MyDrive/Colab Notebooks/data/rides_33.csv'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
weather = pd.read_csv(weather_filepath)
rides = pd.read_csv(rides_path)

In [ ]:
rides.head(2)

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux


In [ ]:
weather.head(2)

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32


XGBoost 출력 레벨 조정
   - 출력을 최소화(대용량 데이터를 다룰때 혹은 메모리 절약을위해서)

In [ ]:
import xgboost as xgb
xgb.set_config(verbosity=0)

EDA
  - 데이터 전처리 및 특성 추출

In [ ]:
# 결측치 확인
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693071 entries, 0 to 693070
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   distance          693071 non-null  float64
 1   cab_type          693071 non-null  object 
 2   time_stamp        693071 non-null  int64  
 3   destination       693071 non-null  object 
 4   source            693071 non-null  object 
 5   price             637976 non-null  float64
 6   surge_multiplier  693071 non-null  float64
 7   id                693071 non-null  object 
 8   product_id        693071 non-null  object 
 9   name              693071 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 52.9+ MB


In [ ]:
rides.columns[rides.isna().sum()>0]

Index(['price'], dtype='object')

In [ ]:
# 결측치가 속한 row들을 확인 - 결측치와 다른 컬럼 데이터들의 관계
rides[rides.isna().any(axis=1)]


,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
18,1.11,Uber,1543673584211,West End,North End,NaN,1.0,fa5fb705-03a0-4eb9-82d9-7fe80872f754,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
31,2.48,Uber,1543794776318,South Station,Beacon Hill,NaN,1.0,eee70d94-6706-4b95-a8ce-0e34f0fa8f37,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
40,2.94,Uber,1543523885298,Fenway,North Station,NaN,1.0,7f47ff53-7cf2-4a6a-8049-83c90e042593,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
60,1.16,Uber,1544731816318,West End,North End,NaN,1.0,43abdbe4-ab9e-4f39-afdc-31cfa375dc25,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
69,2.67,Uber,1543583283653,Beacon Hill,North End,NaN,1.0,80db1c49-9d51-4575-a4f4-1ec23b4d3e31,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
...,...,...,...,...,...,...,...,...,...,...
693025,2.50,Uber,1544886913192,Beacon Hill,South Station,NaN,1.0,70ad9983-92dd-4cdf-af16-732e7e9ee9d0,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
693047,0.91,Uber,1543456028123,Beacon Hill,Haymarket Square,NaN,1.0,84e05ef0-781a-4bd5-a593-bbbf659d4ba0,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
693049,1.79,Uber,1543456028123,Beacon Hill,North End,NaN,1.0,885ef635-299e-45cf-8052-c45a39e9822a,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi
693060,1.61,Uber,1543728484149,Haymarket Square,Theatre District,NaN,1.0,50a7b6be-ed2a-4a11-8d36-73bd977ad66a,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi


In [ ]:
rides.shape

(693071, 10)

In [ ]:
# 결측치를 제거
rides.dropna(inplace=True)

In [ ]:
rides['date'] = pd.to_datetime( rides['time_stamp']*(10**6) )
rides.head(2)

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 09:30:07.890
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 02:00:23.677


In [ ]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637976 entries, 0 to 693070
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   distance          637976 non-null  float64       
 1   cab_type          637976 non-null  object        
 2   time_stamp        637976 non-null  int64         
 3   destination       637976 non-null  object        
 4   source            637976 non-null  object        
 5   price             637976 non-null  float64       
 6   surge_multiplier  637976 non-null  float64       
 7   id                637976 non-null  object        
 8   product_id        637976 non-null  object        
 9   name              637976 non-null  object        
 10  date              637976 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 58.4+ MB


In [ ]:
rides['year'] =  rides['date'].dt.year
rides['month'] = rides['date'].dt.month
rides['day'] = rides['date'].dt.day
rides['hour'] = rides['date'].dt.hour
rides['dayofweek'] = rides['date'].dt.dayofweek

In [ ]:
rides['date'].dt.dayofweek #  2018-12-16   0: 월   5,6 주말
# weekend 라는 컬럼을 만들어서 주말이면 1 그렇지 않으면 0으로 셋팅해 주세요


In [ ]:
rides['weekend'] = rides['dayofweek'].apply(lambda x : 1 if x in [5,6] else 0)

In [ ]:
# 러시아워......범주형 데이터로... 
# 월요일이면서 시간대가 6~9, 15 ~ 18 까지를 rush_hour라고 컬럼을 만들고 값을 1 그렇지 않으면 0

In [ ]:
rides.head(2)

In [ ]:
rides['rush_hour'] =  rides.apply(lambda row : 1 if (row['hour'] in [6,7,8,9,15,16,17,18]) & ( row['dayofweek'] == 0) else 0, axis=1 )

In [ ]:
# 공유플렛폼중에.. 우버 & Lyft

In [ ]:
np.unique(rides['cab_type'])

array(['Lyft', 'Uber'], dtype=object)

'Lyft', 'Uber' 타입의 빈도를 계산해서 cab_freq 컬럼에 저장

In [ ]:
(rides['cab_type'] == 'Uber').sum() / len(rides)

0.5181511530214302

In [ ]:
rides['cab_freq'] = rides.groupby(['cab_type'])['cab_type'].transform('count') / len(rides)

평균인코딩
  - TargetEncoder
  - 레이블에 해당하는 타겟값들의 평균값으로 인코딩

In [ ]:
rides.groupby(['cab_type'])['price'].transform('mean')

0         17.351396
1         17.351396
2         17.351396
3         17.351396
4         17.351396
            ...    
693065    15.795343
693066    15.795343
693067    15.795343
693069    15.795343
693070    15.795343
Name: price, Length: 637976, dtype: float64

In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
from category_encoders.target_encoder import TargetEncoder
encoder = TargetEncoder()
encoder.fit_transform(rides['cab_type'], rides['price'])

,cab_type
0,17.351396
1,17.351396
2,17.351396
3,17.351396
4,17.351396
...,...
693065,15.795343
693066,15.795343
693067,15.795343
693069,15.795343


In [ ]:
rides['cab_type_mean'] = rides.groupby(['cab_type'])['price'].transform('mean')